<a href="https://colab.research.google.com/github/freedom-780/FBI-Firearm-Background-Check/blob/main/Firearm_Background_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id='intro'></a>
## Introduction

> The dataset from The National Instant Criminal Background Check System (NICS) is managed by the FBI  according to the Brady Handgun Violence Prevention Act of 1993 (Brady Act) -- fully implemented on Novemeber 30, 1998 -- and there are differences in state law in regards to requirements. Those that have valid ATF permits and for services and repair picked u by the same person don't require a background check. Data is collected with ATF Form 4473

Types of background checks:

* Handgun—(a) any firearm which has a short stock and is designed to be held and fired by the use of a single hand; and (b) any combination of parts from which a firearm described in paragraph (a) can be assembled.
* Long Gun—a weapon designed or redesigned, made or remade, and intended to be fired from the shoulder, and designed or redesigned and made or remade to use the energy of the explosive in (a) a fixed metallic cartridge to fire a single projectile through a rifled bore for each single pull of the trigger; or (b) a fixed shotgun shell to fire through a smooth bore either a number of ball shot or a single projectile for each single pull of the trigger.
* Other(might want to drop columns or check if they are outliers)—refers to frames, receivers, and other firearms that are neither handguns nor long guns (rifles or shotguns), such as firearms having a pistol grip that expel a shotgun shell, or National Firearms Act firearms, including silencers.

Questions 

* Comparisons of regular gun sales to private gun sales?
* which type of gun sale is increasing the most?
* Any trends between type of gun ownership and mass shootings?(https://github.com/StanfordGeospatialCenter/MSA/tree/master/Data)
* WHat are the trends in background checks and mass shootings for states with the weakest poklicies?
* Are there any intresting overall trends, like spikes in gun ownership for certain periods of time such such as policy changes?
* prices of guns and mass shootings?











In [99]:
# import packages 
import pandas as pd 
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.width', 70)
pd.set_option('display.max_columns', 8)
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline



<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

In [ ]:
# Load data and print out a few lines. Perform operations to inspect data

gun_data = "https://github.com/freedom-780/FBI-Firearm-Background-Check/blob/main/gun_data.xlsx?raw=true"
gun_background_check = pd.read_excel(gun_data)

standford_msa = "https://raw.githubusercontent.com/freedom-780/FBI-Firearm-Background-Check/main/Stanford_MSA_Database.csv"
mass_shootings = pd.read_csv(standford_msa)






In [ ]:
gun_background_check.columns

Index(['month', 'state', 'permit', 'permit_recheck', 'handgun', 'long_gun', 'other',
       'multiple', 'admin', 'prepawn_handgun', 'prepawn_long_gun', 'prepawn_other',
       'redemption_handgun', 'redemption_long_gun', 'redemption_other',
       'returned_handgun', 'returned_long_gun', 'returned_other', 'rentals_handgun',
       'rentals_long_gun', 'private_sale_handgun', 'private_sale_long_gun',
       'private_sale_other', 'return_to_seller_handgun',
       'return_to_seller_long_gun', 'return_to_seller_other', 'totals'],
      dtype='object')

In [ ]:
mass_shootings.columns

Index(['CaseID', 'Title', 'Location', 'City', 'State', 'Latitude', 'Longitude',
       'Number of Civilian Fatalities', 'Number of Civilian Injured',
       'Number of Enforcement Fatalities', 'Number of Enforcement Injured',
       'Total Number of Fatalities', 'Total Number of Victims', 'Description',
       'Date', 'Day of Week', 'Date - Detailed', 'Shooter Name',
       'Number of shooters', 'Shooter Age(s)', 'Average Shooter Age', 'Shooter Sex',
       'Shooter Race', 'Type of Gun - Detailed', 'Type of Gun - General',
       'Number of Shotguns', 'Number of Rifles', 'Number of Handguns',
       'Total Number of Guns', 'Number of Automatic Guns',
       'Number of Semi-Automatic Guns', 'Fate of Shooter at the scene',
       'Fate of Shooter', 'Shooter's Cause of Death', 'School Related',
       'Place Type', 'Relationship to Incident Location',
       'Targeted Victim/s - Detailed', 'Targeted Victim/s - General',
       'Possible Motive - Detailed', 'Possible Motive - General',
   

Disgard the columns not needs for analysis. 

For the background check 'month', 'state, 'handgun', 'long_gun', 'other', 'multiple',private_sale_handgun', 'private_sale_long_gun', 'private_sale_other', 'totals' data only the columns most useful for the analysis are

For mass shooting: 

In [ ]:
# drop columns that are not needed
gun_background_check.drop(gun_background_check.columns.difference(['month', 'state', 'handgun', 'long_gun', 'other', 'multiple', 'private_sale_handgun', 'private_sale_long_gun', 'private_sale_other', 'totals']), 1, inplace=True)

In [ ]:
gun_background_check.columns

Index(['month', 'state', 'handgun', 'long_gun', 'other', 'multiple',
       'private_sale_handgun', 'private_sale_long_gun', 'private_sale_other',
       'totals'],
      dtype='object')

In [ ]:
gun_background_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12485 entries, 0 to 12484
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   month                  12485 non-null  object 
 1   state                  12485 non-null  object 
 2   handgun                12465 non-null  float64
 3   long_gun               12466 non-null  float64
 4   other                  5500 non-null   float64
 5   multiple               12485 non-null  int64  
 6   private_sale_handgun   2750 non-null   float64
 7   private_sale_long_gun  2750 non-null   float64
 8   private_sale_other     2750 non-null   float64
 9   totals                 12485 non-null  int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 975.5+ KB


In [ ]:
mass_shootings.drop(mass_shootings.columns.difference(['State','Date','Type of Gun - General','Number of Shotguns','Number of Rifles','Number of Handguns','Total Number of Guns','Number of Automatic Guns','Number of Semi-Automatic Guns','Possible Motive - General','History of Mental Illness - General']), 1, inplace=True)

In [ ]:
mass_shootings.columns

Index(['State', 'Date', 'Type of Gun - General', 'Number of Shotguns',
       'Number of Rifles', 'Number of Handguns', 'Total Number of Guns',
       'Number of Automatic Guns', 'Number of Semi-Automatic Guns',
       'Possible Motive - General', 'History of Mental Illness - General'],
      dtype='object')

In [ ]:
mass_shootings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   State                                335 non-null    object
 1   Date                                 335 non-null    object
 2   Type of Gun - General                335 non-null    object
 3   Number of Shotguns                   335 non-null    object
 4   Number of Rifles                     335 non-null    object
 5   Number of Handguns                   335 non-null    object
 6   Total Number of Guns                 335 non-null    object
 7   Number of Automatic Guns             335 non-null    object
 8   Number of Semi-Automatic Guns        335 non-null    object
 9   Possible Motive - General            333 non-null    object
 10  History of Mental Illness - General  335 non-null    object
dtypes: object(11)
memory usage: 28.9+ KB


In [ ]:
mass_change_dtype = {'Number of Shotguns': 'float64', 
                     'Number of Rifles': 'float64', 
                     'Total Number of Guns': 'float64',
                     'Number of Automatic Guns': 'float64',
                     'Number of Semi-Automatic Guns': 'float64'
                     }

In [ ]:
mass_shootings.astype(mass_change_dtype)

ValueError: ignored

In [104]:
for i in mass_shootings:
  mass_shootings.drop(mass_shootings.index[mass_shootings[i] == 'Unknown'], inplace = True)
  mass_shootings.drop(mass_shootings.index[mass_shootings[i] == '0 (1)'], inplace = True)


In [107]:
mass_shootings = mass_shootings.astype(mass_change_dtype)

In [108]:
mass_shootings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 333
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State                                146 non-null    object 
 1   Date                                 146 non-null    object 
 2   Type of Gun - General                146 non-null    object 
 3   Number of Shotguns                   146 non-null    float64
 4   Number of Rifles                     146 non-null    float64
 5   Number of Handguns                   146 non-null    object 
 6   Total Number of Guns                 146 non-null    float64
 7   Number of Automatic Guns             146 non-null    float64
 8   Number of Semi-Automatic Guns        146 non-null    float64
 9   Possible Motive - General            146 non-null    object 
 10  History of Mental Illness - General  146 non-null    object 
dtypes: float64(5), object(6)
memory 

for the mass shootings database, the following colu

In [ ]:
# print statistics about permit data 

print(f"Shape of permit data: {gun_background_check.shape}")
print(f"Number of mission values in permit data: \
{gun_background_check.isnull().sum().sum()}")
print(f"Number of duplicate values in permit data: \
{gun_background_check.duplicated().sum()}")

Shape of permit data: (12485, 10)
Number of mission values in permit data: 36229
Number of duplicate values in permit data: 0


In [ ]:
gun_background_check.isnull().sum()

month                       0
state                       0
handgun                    20
long_gun                   19
other                    6985
multiple                    0
private_sale_handgun     9735
private_sale_long_gun    9735
private_sale_other       9735
totals                      0
dtype: int64

In [ ]:
# print statistics about mass shootings data

print(f"Shape of mass shooting data: {mass_shootings.shape}")
print(f"Number of mission values in mass shooting data: \
{mass_shootings.isnull().sum().sum()}")
print(f"Number of duplicate values in mass shooting data: \
{mass_shootings.duplicated().sum()}")

Shape of mass shooting data: (335, 11)
Number of mission values in mass shooting data: 2
Number of duplicate values in mass shooting data: 1


In [ ]:
mass_shootings.isnull().any()

State                                  False
Date                                   False
Type of Gun - General                  False
Number of Shotguns                     False
Number of Rifles                       False
Number of Handguns                     False
Total Number of Guns                   False
Number of Automatic Guns               False
Number of Semi-Automatic Guns          False
Possible Motive - General               True
History of Mental Illness - General    False
dtype: bool

In [ ]:
mass_shootings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 55 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   CaseID                                335 non-null    int64  
 1   Title                                 335 non-null    object 
 2   Location                              335 non-null    object 
 3   City                                  335 non-null    object 
 4   State                                 335 non-null    object 
 5   Latitude                              335 non-null    float64
 6   Longitude                             335 non-null    float64
 7   Number of Civilian Fatalities         335 non-null    int64  
 8   Number of Civilian Injured            335 non-null    int64  
 9   Number of Enforcement Fatalities      335 non-null    int64  
 10  Number of Enforcement Injured         335 non-null    int64  
 11  Total Number of Fat

In [ ]:
mass_shootings_data.columns

Index(['CaseID', 'Title', 'Location', 'City', 'State', 'Latitude', 'Longitude',
       'Number of Civilian Fatalities', 'Number of Civilian Injured',
       'Number of Enforcement Fatalities', 'Number of Enforcement Injured',
       'Total Number of Fatalities', 'Total Number of Victims', 'Description',
       'Date', 'Day of Week', 'Date - Detailed', 'Shooter Name',
       'Number of shooters', 'Shooter Age(s)', 'Average Shooter Age',
       'Shooter Sex', 'Shooter Race', 'Type of Gun - Detailed',
       'Type of Gun - General', 'Number of Shotguns', 'Number of Rifles',
       'Number of Handguns', 'Total Number of Guns',
       'Number of Automatic Guns', 'Number of Semi-Automatic Guns',
       'Fate of Shooter at the scene', 'Fate of Shooter',
       'Shooter's Cause of Death', 'School Related', 'Place Type',
       'Relationship to Incident Location', 'Targeted Victim/s - Detailed',
       'Targeted Victim/s - General', 'Possible Motive - Detailed',
       'Possible Motive - Genera

In [ ]:
us_census_data.columns


Index(['Fact', 'Fact Note', 'Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida',
       'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
       'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype='object')

In [ ]:
permit_data.head(10)


,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,prepawn_long_gun,prepawn_other,redemption_handgun,redemption_long_gun,redemption_other,returned_handgun,returned_long_gun,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
0,2017-09,Alabama,16717.0,0.0,5734.0,6320.0,221.0,317,0.0,15.0,21.0,2.0,1378.0,1262.0,1.0,0.0,0.0,0.0,0.0,0.0,9.0,16.0,3.0,0.0,0.0,3.0,32019
1,2017-09,Alaska,209.0,2.0,2320.0,2930.0,219.0,160,0.0,5.0,2.0,0.0,200.0,154.0,2.0,28.0,30.0,0.0,0.0,0.0,17.0,24.0,1.0,0.0,0.0,0.0,6303
2,2017-09,Arizona,5069.0,382.0,11063.0,7946.0,920.0,631,0.0,13.0,6.0,0.0,1474.0,748.0,3.0,82.0,5.0,0.0,0.0,0.0,38.0,12.0,2.0,0.0,0.0,0.0,28394
3,2017-09,Arkansas,2935.0,632.0,4347.0,6063.0,165.0,366,51.0,12.0,13.0,0.0,1296.0,1824.0,4.0,0.0,0.0,0.0,0.0,0.0,13.0,23.0,0.0,0.0,2.0,1.0,17747
4,2017-09,California,57839.0,0.0,37165.0,24581.0,2984.0,0,0.0,0.0,0.0,0.0,535.0,397.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123506
5,2017-09,Colorado,4356.0,0.0,15751.0,13448.0,1007.0,1062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,202.0,46.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35873
6,2017-09,Connecticut,4343.0,673.0,4834.0,1993.0,274.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12117
7,2017-09,Delaware,275.0,0.0,1414.0,1538.0,66.0,68,0.0,0.0,1.0,0.0,26.0,16.0,3.0,0.0,0.0,0.0,0.0,0.0,55.0,34.0,3.0,1.0,2.0,0.0,3502
8,2017-09,District of Columbia,1.0,0.0,56.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61
9,2017-09,Florida,10784.0,0.0,39199.0,17949.0,2319.0,1721,1.0,18.0,7.0,0.0,3657.0,1416.0,6.0,264.0,28.0,0.0,0.0,0.0,11.0,9.0,0.0,0.0,1.0,0.0,77390


In [ ]:
print(f"Shape of Permit Data: {permit_data.shape} ")


Shape of Permit Data: (12485, 27) 


In [ ]:
permit_data.describe().T

,count,mean,std,min,25%,50%,75%,max
permit,12461.0,6413.629404,23752.338269,0.0,0.00,518.0,4272.00,522188.0
permit_recheck,1100.0,1165.956364,9224.200609,0.0,0.00,0.0,0.00,116681.0
handgun,12465.0,5940.881107,8618.584060,0.0,865.00,3059.0,7280.00,107224.0
long_gun,12466.0,7810.847585,9309.846140,0.0,2078.25,5122.0,10380.75,108058.0
other,5500.0,360.471636,1349.478273,0.0,17.00,121.0,354.00,77929.0
multiple,12485.0,268.603364,783.185073,0.0,15.00,125.0,301.00,38907.0
admin,12462.0,58.898090,604.814818,0.0,0.00,0.0,0.00,28083.0
prepawn_handgun,10542.0,4.828021,10.907756,0.0,0.00,0.0,5.00,164.0
prepawn_long_gun,10540.0,7.834156,16.468028,0.0,0.00,1.0,8.00,269.0
prepawn_other,5115.0,0.165591,1.057105,0.0,0.00,0.0,0.00,49.0


May want to change colums labels for U.S. census data and make it more concises  

In [ ]:
permit_data

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,prepawn_long_gun,prepawn_other,redemption_handgun,redemption_long_gun,redemption_other,returned_handgun,returned_long_gun,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
0,2017-09,Alabama,16717.0,0.0,5734.0,6320.0,221.0,317,0.0,15.0,21.0,2.0,1378.0,1262.0,1.0,0.0,0.0,0.0,0.0,0.0,9.0,16.0,3.0,0.0,0.0,3.0,32019
1,2017-09,Alaska,209.0,2.0,2320.0,2930.0,219.0,160,0.0,5.0,2.0,0.0,200.0,154.0,2.0,28.0,30.0,0.0,0.0,0.0,17.0,24.0,1.0,0.0,0.0,0.0,6303
2,2017-09,Arizona,5069.0,382.0,11063.0,7946.0,920.0,631,0.0,13.0,6.0,0.0,1474.0,748.0,3.0,82.0,5.0,0.0,0.0,0.0,38.0,12.0,2.0,0.0,0.0,0.0,28394
3,2017-09,Arkansas,2935.0,632.0,4347.0,6063.0,165.0,366,51.0,12.0,13.0,0.0,1296.0,1824.0,4.0,0.0,0.0,0.0,0.0,0.0,13.0,23.0,0.0,0.0,2.0,1.0,17747
4,2017-09,California,57839.0,0.0,37165.0,24581.0,2984.0,0,0.0,0.0,0.0,0.0,535.0,397.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12480,1998-11,Virginia,0.0,NaN,14.0,2.0,NaN,8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
12481,1998-11,Washington,1.0,NaN,65.0,286.0,NaN,8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,361
12482,1998-11,West Virginia,3.0,NaN,149.0,251.0,NaN,5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,408
12483,1998-11,Wisconsin,0.0,NaN,25.0,214.0,NaN,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241


In [ ]:
us_census_data.head()

,Fact,Fact Note,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,"Population estimates, July 1, 2016, (V2016)",NaN,"4,863,300","741,894","6,931,071","2,988,248","39,250,017","5,540,545","3,576,452","952,065","20,612,439","10,310,371","1,428,557","1,683,140","12,801,539","6,633,053","3,134,693","2,907,289","4,436,974","4,681,666","1,331,479","6,016,447","6,811,779","9,928,300","5,519,952","2,988,726","6,093,000","1,042,520","1,907,116","2,940,058","1,334,795","8,944,469",2081015,19745289,10146788,757952,11614373,3923561,4093465,12784227,1056426,4961119,865454,6651194,"27,862,596","3,051,217","624,594","8,411,808","7,288,000","1,831,102","5,778,708","585,501"
1,"Population estimates base, April 1, 2010, (V2...",NaN,"4,780,131","710,249","6,392,301","2,916,025","37,254,522","5,029,324","3,574,114","897,936","18,804,592","9,688,680","1,360,301","1,567,650","12,831,574","6,484,136","3,046,869","2,853,129","4,339,344","4,533,479","1,328,364","5,773,786","6,547,813","9,884,129","5,303,924","2,968,103","5,988,928","989,414","1,826,334","2,700,691","1,316,461","8,791,953",2059198,19378110,9535688,672591,11536727,3751615,3831072,12702857,1052940,4625410,814195,6346298,"25,146,100","2,763,888","625,741","8,001,041","6,724,545","1,853,011","5,687,289","563,767"
2,"Population, percent change - April 1, 2010 (es...",NaN,1.70%,4.50%,8.40%,2.50%,5.40%,10.20%,0.10%,6.00%,9.60%,6.40%,5.00%,7.40%,-0.20%,2.30%,2.90%,1.90%,2.20%,3.30%,0.20%,4.20%,4.00%,0.40%,4.10%,0.70%,1.70%,5.40%,4.40%,8.90%,1.40%,1.70%,0.011,0.019,0.064,0.127,0.007,0.046,0.068,0.006,0.003,0.073,0.063,0.048,10.80%,10.40%,-0.20%,5.10%,8.40%,-1.20%,1.60%,3.90%
3,"Population, Census, April 1, 2010",NaN,"4,779,736","710,231","6,392,017","2,915,918","37,253,956","5,029,196","3,574,097","897,934","18,801,310","9,687,653","1,360,301","1,567,582","12,830,632","6,483,802","3,046,355","2,853,118","4,339,367","4,533,372","1,328,361","5,773,552","6,547,629","9,883,640","5,303,925","2,967,297","5,988,927","989,415","1,826,341","2,700,551","1,316,470","8,791,894",2059179,19378102,9535483,672591,11536504,3751351,3831074,12702379,1052567,4625364,814180,6346105,"25,145,561","2,763,885","625,741","8,001,024","6,724,540","1,852,994","5,686,986","563,626"
4,"Persons under 5 years, percent, July 1, 2016, ...",NaN,6.00%,7.30%,6.30%,6.40%,6.30%,6.10%,5.20%,5.80%,5.50%,6.40%,6.40%,6.80%,6.00%,6.40%,6.40%,6.70%,6.20%,6.60%,4.90%,6.10%,5.30%,5.80%,6.40%,6.30%,6.10%,6.00%,7.00%,6.30%,4.80%,5.80%,0.062,0.059,0.06,0.073,0.06,0.068,0.058,0.056,0.052,0.059,0.071,0.061,7.20%,8.30%,4.90%,6.10%,6.20%,5.50%,5.80%,6.50%


May want to change some of the int data types to floats

In [ ]:
permit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12485 entries, 0 to 12484
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   month                      12485 non-null  object 
 1   state                      12485 non-null  object 
 2   permit                     12461 non-null  float64
 3   permit_recheck             1100 non-null   float64
 4   handgun                    12465 non-null  float64
 5   long_gun                   12466 non-null  float64
 6   other                      5500 non-null   float64
 7   multiple                   12485 non-null  int64  
 8   admin                      12462 non-null  float64
 9   prepawn_handgun            10542 non-null  float64
 10  prepawn_long_gun           10540 non-null  float64
 11  prepawn_other              5115 non-null   float64
 12  redemption_handgun         10545 non-null  float64
 13  redemption_long_gun        10544 non-null  flo

In [ ]:
permit_data[permit_data.isnull()].count

<bound method DataFrame.count of       month state  ...  return_to_seller_other  totals
0       NaN   NaN  ...                     NaN     NaN
1       NaN   NaN  ...                     NaN     NaN
2       NaN   NaN  ...                     NaN     NaN
3       NaN   NaN  ...                     NaN     NaN
4       NaN   NaN  ...                     NaN     NaN
...     ...   ...  ...                     ...     ...
12480   NaN   NaN  ...                     NaN     NaN
12481   NaN   NaN  ...                     NaN     NaN
12482   NaN   NaN  ...                     NaN     NaN
12483   NaN   NaN  ...                     NaN     NaN
12484   NaN   NaN  ...                     NaN     NaN

[12485 rows x 27 columns]>

The us census dtypes are obejcts meaning strings which means they need to change from strings to floats

In [ ]:
us_census_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Fact            80 non-null     object
 1   Fact Note       28 non-null     object
 2   Alabama         65 non-null     object
 3   Alaska          65 non-null     object
 4   Arizona         65 non-null     object
 5   Arkansas        65 non-null     object
 6   California      65 non-null     object
 7   Colorado        65 non-null     object
 8   Connecticut     65 non-null     object
 9   Delaware        65 non-null     object
 10  Florida         65 non-null     object
 11  Georgia         65 non-null     object
 12  Hawaii          65 non-null     object
 13  Idaho           65 non-null     object
 14  Illinois        65 non-null     object
 15  Indiana         65 non-null     object
 16  Iowa            65 non-null     object
 17  Kansas          65 non-null     object
 18  Kentucky    

> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning (Replace this with more specific notes!)

In [ ]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.


In [ ]:
print(f"Shape of data: {permit_data.shape}")
print(f"Number of missing values in the data:\
{permit_data.isnull().sum().sum()}")
print(f"Number of duplicated values: {permit_data.duplicated().sum()}")

Shape of data: (12485, 27)
Number of missing values in the data:154595
Number of duplicated values: 0


Remove missing values 

In [ ]:
permit_data.dropna(inplace=True)

# Verify missing values 

print(f"Number of missing values in the data:\
{permit_data.isnull().sum().sum()}")

Number of missing values in the data:0


In [ ]:
print(f"Shape of data: {us_census_data.shape}")
print(f"Number of missing values in the data:\
{us_census_data.isnull().sum().sum()}")
print(f"Number of duplicated values: {us_census_data.duplicated().sum()}")

Shape of data: (85, 52)
Number of missing values in the data:1062
Number of duplicated values: 3


In [ ]:
permit_data.duplicated().sum()

0

In [ ]:
us_census_data.isnull().sum()

Fact               5
Fact Note         57
Alabama           20
Alaska            20
Arizona           20
Arkansas          20
California        20
Colorado          20
Connecticut       20
Delaware          20
Florida           20
Georgia           20
Hawaii            20
Idaho             20
Illinois          20
Indiana           20
Iowa              20
Kansas            20
Kentucky          20
Louisiana         20
Maine             20
Maryland          20
Massachusetts     20
Michigan          20
Minnesota         20
Mississippi       20
Missouri          20
Montana           20
Nebraska          20
Nevada            20
New Hampshire     20
New Jersey        20
New Mexico        20
New York          20
North Carolina    20
North Dakota      20
Ohio              20
Oklahoma          20
Oregon            20
Pennsylvania      20
Rhode Island      20
South Carolina    20
South Dakota      20
Tennessee         20
Texas             20
Utah              20
Vermont           20
Virginia     

In [ ]:
us_census_data.duplicated().sum()

3

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# What census data ismost associated withhigh gun per capita?Which states have hadthe highest growth ingun registrations?What is the overalltrend of gunpurchases?


### Research Question 2  (Replace this header name!)
#What census data ismost associated withhigh gun per capita?Which states have hadthe highest growth ingun registrations?What is the overalltrend of gunpurchases?

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!